In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-prediction-on-movie-reviews/movies.csv
/kaggle/input/sentiment-prediction-on-movie-reviews/sample.csv
/kaggle/input/sentiment-prediction-on-movie-reviews/train.csv
/kaggle/input/sentiment-prediction-on-movie-reviews/test.csv


In [2]:
import numpy as np 
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [3]:
mtrain = pd.read_csv("/kaggle/input/sentiment-prediction-on-movie-reviews/train.csv")
movies = pd.read_csv("/kaggle/input/sentiment-prediction-on-movie-reviews/movies.csv")

In [4]:
mtrain.shape, movies.shape

((162758, 5), (143258, 14))

In [5]:
train_df = mtrain.drop(['movieid', 'reviewerName', 'isFrequentReviewer'], axis = 1)

In [6]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
train_df = train_df.fillna('')
x = train_df['reviewText']
y = train_df['sentiment']
X_train, X_test, y_train, y_test =  train_test_split(x, y, test_size = 0.33, random_state = 42)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
tfidf = TfidfVectorizer(token_pattern = r'[a-zA-Z]+')
X_train_vector = tfidf.fit_transform(X_train)
X_test_vector = tfidf.transform(X_test)

In [8]:
from sklearn.model_selection import GridSearchCV
parameters = {'solver' : ['liblinear', 'lbfgs', 'newton-cg'], 'penalty' : ['none','l1', 'l2'], 'C' : [0.01, 0.1, 1,10, 20, 50, 100], 'max_iter': [100, 200, 300]}
LR = LogisticRegression(random_state = 1)
Grid_SearchCV = GridSearchCV(estimator = LR, param_grid = parameters, n_jobs=-1, cv=3, scoring='accuracy')
Grid_SearchCV

GridSearchCV(cv=3, estimator=LogisticRegression(random_state=1), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 20, 50, 100],
                         'max_iter': [100, 200, 300],
                         'penalty': ['none', 'l1', 'l2'],
                         'solver': ['liblinear', 'lbfgs', 'newton-cg']},
             scoring='accuracy')

In [9]:
LR_result = Grid_SearchCV.fit(X_train_vector,y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour

In [10]:
print('Best Parameters:', Grid_SearchCV.best_params_)
print('Best Score:', Grid_SearchCV.best_score_)

Best Parameters: {'C': 10, 'max_iter': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
Best Score: 0.7909066732693243


In [11]:
LR_model = Grid_SearchCV.best_estimator_
LR_model.fit(X_train_vector,y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=10, random_state=1)

In [12]:
y_pred = LR_model.predict(X_test_vector)
accuracy = accuracy_score(y_test,y_pred)
print("LR_Accuracy: ", accuracy)

LR_Accuracy:  0.7952747109530636


In [13]:
test = pd.read_csv("/kaggle/input/sentiment-prediction-on-movie-reviews/test.csv")

In [14]:
test_df = test.drop(['movieid', 'reviewerName', 'isTopCritic'], axis = 1)
test_df['reviewText'] = test_df['reviewText'].fillna('')

In [15]:
Z_test = test_df['reviewText']

In [16]:
test_df_vector = tfidf.transform(Z_test)

In [17]:
test_predictions = LR_model.predict(test_df_vector)

In [18]:
submission = pd.DataFrame(columns = ['id', 'sentiment'])
submission['id'] = [i for i in range(len(test_predictions))]
submission['sentiment'] = test_predictions

In [19]:
submission.shape, submission.info

((55315, 2),
 <bound method DataFrame.info of           id sentiment
 0          0  POSITIVE
 1          1  POSITIVE
 2          2  POSITIVE
 3          3  POSITIVE
 4          4  POSITIVE
 ...      ...       ...
 55310  55310  POSITIVE
 55311  55311  NEGATIVE
 55312  55312  NEGATIVE
 55313  55313  POSITIVE
 55314  55314  NEGATIVE
 
 [55315 rows x 2 columns]>)

In [20]:
submission.head()

,id,sentiment
0,0,POSITIVE
1,1,POSITIVE
2,2,POSITIVE
3,3,POSITIVE
4,4,POSITIVE


In [21]:
submission.to_csv('submission.csv', index = False)